In [1]:
# pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np

experiment = pybamm.Experiment(
    [
        ("Charge at C/3 until 4.2 V",
        "Hold at 4.2 V until C/20",),
    ]
)

model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot(["Voltage [V]",'Current [A]'])

At t = 205.194, , mxstep steps taken before reaching tout.
At t = 205.194 and h = 4.10627e-13, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0677153228953746, step=0.01067715322895374…